In [9]:
import pandas as pd
import numpy as np
import warnings
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

In [83]:
portfolio = ['ADBE', 'ADP', 'AMGN', 'AVGO', 'BMY', 'CHTR',
       'CMCSA', 'COST', 'EA', 'GILD', 'HD', 'IBM', 'INTU', 'KO', 'LIN',
       'LMT', 'MA', 'MCD', 'MDLZ', 'MMM', 'MO', 'MRK', 'NKE', 'TMO', 'TSLA']
len(portfolio)

25

# func

In [78]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:[float(element) for element in x])
  tp_data['state_day_start'] = tp_data['state_day_start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tp_data['state_day_end'] = tp_data['state_day_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

  spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
  spread_df['spread'] = spread_df['spread'] / 2
  s = np.setdiff1d(tp_data.state_ticker.to_numpy(), spread_df.ticker.to_numpy())
  tp_data['spread'] = tp_data.state_ticker.loc[~tp_data.state_ticker.isin(s)].apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
  tp_data['spread'].loc[tp_data.state_ticker.isin(s)] = 5
  tp_data['deviation'] = 1
  tp_data['point'] = tp_data.state_ticker.loc[~tp_data.state_ticker.isin(s)].apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
  tp_data['point'].loc[tp_data.state_ticker.isin(s)] = 0.01
  tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
  tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput0.csv')

In [77]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [76]:
def get_stock_data(tp_data, min_thresh = 8, cut_thresh = 0.61):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , min_thresh , cut_thresh) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max())
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

# TP data

In [8]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput270.csv')
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-05-16 13:50:00,2022-05-18 13:50:00,90.985,90.510,NaN,7.400000,3.610526,3.884211,3.610526,"[-0.13684210526312074, -0.26315789473680745, -..."
1,MRK,2,2022-05-18 18:50:00,2022-05-19 19:00:00,92.204,92.394,NaN,5.915789,NaN,NaN,-1.000000,"[0.2842105263158036, 0.6526315789474258, 1.178..."
2,MRK,1,2022-05-25 16:10:00,2022-05-26 14:20:00,93.138,92.818,NaN,2.225000,0.131250,NaN,-1.000000,"[0.44374999999999665, 0.4750000000000133, 0.13..."
3,MRK,2,2022-05-26 14:35:00,2022-06-01 19:35:00,93.028,93.458,NaN,6.902326,3.600000,5.506977,3.600000,"[-0.05116279069765351, -0.5860465116278868, -0..."
4,MRK,2,2022-06-01 19:30:00,2022-06-03 13:40:00,91.385,92.145,NaN,3.480263,-0.125000,NaN,1.388158,"[-0.006578947368433847, -0.1250000000000187, 0..."


In [374]:
a = None
if a:
    print('a')

In [371]:
data1 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
data2 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
data1['spread2'] = data1.ticker.apply(lambda x: data2.spread.loc[data2.ticker == x].iloc[0])
data1.loc[data1.spread != data1.spread2]

,Unnamed: 0,ticker,point,spread,deviation,spread2
0,0,AAPL,0.01,6,5,9
5,5,AMD,0.01,10,5,25
8,8,AMZN,0.01,4,5,6
12,12,BAC,0.01,8,5,9
13,13,BA,0.01,24,5,46
24,24,DE30,0.10,39,5,36
28,28,FB,0.01,8,5,41
38,38,JP225,0.10,90,5,89
50,50,MSFT,0.01,10,5,21
52,52,NFLX,0.01,24,5,81


In [7]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,634.000000,634.000000,634.000000,634.000000,634.000000
mean,0.133611,2.637146,0.020554,0.076891,-0.011929
std,3.938061,5.902176,3.700856,3.054069,3.709052
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.035287,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.538088,-1.000000,-1.000000,-1.000000
75%,-1.000000,2.808854,-1.000000,-1.000000,-1.000000
max,38.100000,52.037500,38.100000,33.528571,38.100000


In [260]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     46.891751
state_max_r             985.999242
state_MA_r               -9.228510
state_RSI_r              20.969008
last_r                  -16.195593
dtype: float64

In [266]:
tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
tp_data.estimate_with_thresh.sum()

-63.334356909509566

In [264]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."


# concat data

In [79]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/*.csv"):
    tp_data = preprocess_tp_data(path)
    tp_data['bash_path'] = str(os.path.basename(path))
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)


In [80]:
cut_thresh = 0.61
min_thresh = 6
concat_data['estimate_with_thresh'] = concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , min_thresh , cut_thresh) )
concat_data['estimate_with_thresh2'] = concat_data.state_chain_new.apply(lambda x : x.max() * cut_thresh )
concat_data.estimate_with_thresh2.loc[concat_data.estimate_with_thresh2 < 3] = -1
concat_data.loc[concat_data.state_ticker.isin(portfolio)].estimate_with_thresh.sum()

670.5873365625287

In [81]:
stock_tp_data = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].groupby(['state_ticker' , 'bash_path']).sum()
stock_tp_data['win_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r > 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data['loss_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r < 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data.win_count.fillna(0 , inplace = True)
stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 
stock_tp_data

state_close_estimate  state_max_r  \
state_ticker bash_path                                               
AAPL         stateoutput0.csv              -14.000000    14.001688   
             stateoutput120.csv             -0.473267    24.204790   
             stateoutput150.csv             -2.117733    21.692807   
             stateoutput180.csv             -4.709986    58.150310   
             stateoutput210.csv            -13.768421    27.340992   
...                                               ...          ...   
XPEV         stateoutput30.csv              -3.800000     6.290476   
YUMC         stateoutput0.csv               -3.368750     5.113889   
             stateoutput30.csv              -6.000000    11.413559   
ZTO          stateoutput0.csv               -0.519298    17.797734   
             stateoutput30.csv              -7.000000    13.482468   

                                 state_MA_r  state_RSI_r     last_r  \
state_ticker bash_path                                                
AAPL         stateoutput0.csv    -14.000000    -7.166667 -14.000000   
             stateoutput120.csv   -0.473267     2.783911  -0.473267   
             stateoutput150.csv   -1.678667    -2.117733  -2.117733   
             stateoutput180.csv    3.864214     2.939569   2.939569   
             stateoutput210.csv  -11.968672   -10.012171 -13.768421   
...                                     ...          ...        ...   
XPEV         stateoutput30.csv    -3.800000    -3.800000  -3.800000   
YUMC         stateoutput0.csv     -2.553365    -3.368750  -3.368750   
             stateoutput30.csv    -6.000000    -6.000000  -6.000000   
ZTO          stateoutput0.csv     -0.519298    -0.519298  -0.519298   
             stateoutput30.csv    -6.700000    -4.218182  -7.000000   

                                 estimate_with_thresh  estimate_with_thresh2  \
state_ticker bash_path                                                         
AAPL         stateoutput0.csv              -14.000000             -14.000000   
             stateoutput120.csv              1.792516               1.792516   
             stateoutput150.csv             -2.304192              -0.352847   
             stateoutput180.csv              3.170835               5.643230   
             stateoutput210.csv             -9.011765              -3.152352   
...                                               ...                    ...   
XPEV         stateoutput30.csv              -3.909091              -5.000000   
YUMC         stateoutput0.csv               -3.468421              -5.000000   
             stateoutput30.csv              -6.000000              -6.000000   
ZTO          stateoutput0.csv               -1.375346              -3.323733   
             stateoutput30.csv              -8.000000              -8.000000   

                                 win_count  loss_count  
state_ticker bash_path                                  
AAPL         stateoutput0.csv            0          14  
             stateoutput120.csv          2           8  
             stateoutput150.csv          2          10  
             stateoutput180.csv          2          15  
             stateoutput210.csv          1          16  
...                                    ...         ...  
XPEV         stateoutput30.csv           1           4  
YUMC         stateoutput0.csv            1           4  
             stateoutput30.csv           0           6  
ZTO          stateoutput0.csv            2           6  
             stateoutput30.csv           0           7  

[753 rows x 9 columns]

In [84]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].groupby('state_ticker').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,,,,,,,,,
ADBE,245.733843,454.073596,219.542219,50.464330,217.135973,194.365300,162.676768,12,71
ADP,19.371441,182.837133,10.269597,-4.180440,11.825427,13.260849,13.364826,12,62
AMGN,88.005697,290.336531,71.197741,51.462124,71.784292,69.667294,67.237293,18,60
AVGO,20.056106,197.680921,10.609004,19.712367,6.324333,31.415537,22.317700,11,59
BMY,30.929567,246.899391,-1.484876,17.841006,14.667181,4.175253,2.670658,19,72
CHTR,24.858028,320.181959,128.701917,28.919567,124.744284,-0.706443,80.269316,15,73
CMCSA,-6.975889,162.152991,-5.558410,4.614089,-7.915889,-6.916813,-11.401883,9,63
COST,-1.838396,169.942450,1.346952,-2.822555,-2.229568,0.480528,-0.645220,10,60
EA,14.942419,166.305472,-11.151196,16.647508,-8.707310,4.343949,-7.675647,11,71


In [86]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].groupby('bash_path').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
bash_path,,,,,,,,,
stateoutput0.csv,-43.637030,332.319021,-53.683860,-35.490941,-60.342622,-61.223055,-57.044823,21,163
stateoutput120.csv,53.168864,567.034121,51.137292,41.896451,60.160589,64.786578,38.688546,32,169
stateoutput150.csv,351.077287,956.632136,450.232982,78.192888,448.117025,267.726682,259.909777,36,155
stateoutput180.csv,64.877738,589.721381,66.918450,75.410406,65.281370,43.529344,68.671368,42,158
stateoutput210.csv,114.247821,622.859100,93.634749,161.172693,92.067595,109.227398,119.733731,36,160
stateoutput240.csv,17.990097,425.463037,-24.084825,19.031226,-20.287003,4.516499,7.339785,23,159
stateoutput270.csv,57.496907,522.687796,35.694857,13.017407,31.514410,40.204919,8.113788,38,182
stateoutput30.csv,60.375411,622.726126,119.893670,5.249210,122.791678,26.748099,86.763625,26,158
stateoutput60.csv,118.259919,605.520716,69.648154,103.090212,59.909628,93.814450,81.128784,39,177


In [125]:
sub_concat_data = concat_data.loc[(concat_data.state_ticker.isin(portfolio))]

def get_return(min_thresh, cut_thresh):
    sub_concat_data.estimate_with_thresh = sub_concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , min_thresh , cut_thresh) )
    return sub_concat_data.estimate_with_thresh.sum()

min_thresh_list = np.arange(6.0,15.0,0.5)
cut_thresh_list =np.arange(0.5,0.8,0.01)
grid_list = []
for i in min_thresh_list:
    for j in cut_thresh_list:
        grid_list.append([i,j])
max = 0
i_temp = 0
j_temp = 0
for x in grid_list:
    temp = get_return(x[0],x[1])
    if(temp > max):
        max = temp
        i_temp = x[0]
        j_temp = x[1]

print(max, i_temp, j_temp)



545.9006094719391 6.0 0.6100000000000001


In [88]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio) & (stock_tp_data.index.get_level_values(1) == 'stateoutput0.csv')]

,,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,bash_path,,,,,,,,,
ADBE,stateoutput0.csv,-5.000000,3.884320,-4.871761,-5.000000,-5.000000,-5.000000,-5.000000,0,5
ADP,stateoutput0.csv,-7.000000,6.061252,-6.754839,-7.000000,-7.000000,-7.000000,-7.000000,0,7
AMGN,stateoutput0.csv,-1.439556,18.990263,-2.548148,-2.548148,-2.548148,-4.934269,-1.456358,1,10
AVGO,stateoutput0.csv,-3.622222,-0.286229,-3.622222,-3.622222,-3.622222,-3.867005,-6.000000,1,5
BMY,stateoutput0.csv,-9.185714,16.010420,-8.919048,-2.006924,-9.185714,-9.537500,-8.648812,1,12
CHTR,stateoutput0.csv,3.236364,32.124486,4.158398,3.236364,3.236364,-5.516152,0.286966,1,10
CMCSA,stateoutput0.csv,-6.000000,3.512900,-6.000000,-6.000000,-6.000000,-6.000000,-6.000000,0,6
COST,stateoutput0.csv,-5.000000,3.952306,-4.227586,-5.000000,-5.000000,-5.000000,-5.000000,0,5
EA,stateoutput0.csv,6.406354,21.728975,6.732440,6.406354,6.406354,5.881126,2.028210,3,5


# Stock_tp_data

In [46]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max() * 0.75)
    tp_data['estimate_with_thresh2'].loc[tp_data['estimate_with_thresh2'] < 4] = -1
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

In [47]:
stock_tp_data = get_stock_data(concat_data)
stock_tp_data.sum()

state_close_estimate        19.951226
state_max_r              15806.710344
state_MA_r                -374.072617
state_RSI_r               -143.036494
last_r                    -491.838738
estimate_with_thresh      -967.849670
estimate_with_thresh2     -546.276837
win_count                  981.000000
loss_count                5697.000000
dtype: float64

In [48]:
portfolio_list = stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10].index

In [615]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput150.csv'
tp_data = preprocess_tp_data(path)
#tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
stock_tp_data = get_stock_data(tp_data)
stock_tp_data.sum()

state_close_estimate      255.321413
state_max_r              2018.145404
state_MA_r                322.612335
state_RSI_r                 3.843986
last_r                    321.780874
estimate_with_thresh      152.794267
estimate_with_thresh2     274.091012
win_count                 103.000000
loss_count                579.000000
dtype: float64

In [105]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7, cut_max_default = 7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        list_temp2 = [x[0] * (1 - cut_thresh)]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            temp2 = x[0:i].max() * (1 - cut_thresh)
            list_temp.append(temp)
            list_temp2.append(temp2)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        temp_df['c'] = list_temp2
        temp_df.b.loc[temp_df.c > cut_max_default] =  temp_df.a - cut_max_default
        # print(temp_df.iloc[50:]
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [7]:
a = {'a':1}
'a' in a.keys() 

True

In [88]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput150.csv'
tp_data = preprocess_tp_data(path)

tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.8) )
a = tp_data.loc[(tp_data.estimate_with_thresh > 30)].iloc[2]
a.state_chain_new , a.estimate_with_thresh , a.state_chain_new.max() * 0.8

(array([ 0.49474535,  0.97978981,  1.37590946,  2.07113985,  1.65885206,
         2.087308  ,  2.8148747 ,  3.76879547,  3.59902991,  4.18916734,
         2.87954729,  2.8148747 ,  3.04931285,  2.02263541,  2.63702506,
         2.45109135,  2.68552951,  2.60468876,  2.97655618,  3.71220695,
         2.58043654,  1.49717057,  2.00646726,  2.16006467,  1.43249798,
         2.00646726,  2.41875505,  1.24656427,  1.09296686,  1.62651576,
         1.82053355,  2.28940986,  2.66936136,  2.26515764,  1.87712207,
         1.92562652,  1.57801132,  1.61034762,  2.61277284,  2.51576395,
         2.04688763,  1.68310428,  1.09296686,  1.47291835,  1.4890865 ,
         0.69684721,  0.09054163,  0.33306386,  0.1956346 ,  0.61600647,
         0.51899757,  0.68876314,  0.72918351,  0.94745352,  1.27890057,
         1.38399353,  1.64268391,  1.90945837,  2.04688763,  3.1301536 ,
         2.8148747 ,  2.93613581,  3.38884398,  2.97655618,  2.79062247,
         2.75828618,  2.63702506,  2.66936136,  2.6

In [249]:
#tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
def get_return_with_max_estimate(state_return_list, min_thresh = 12, cut_thresh = 0.66):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

tp_data['new_estimate'] = tp_data.state_chain.apply(lambda x: get_return_with_max_estimate(x, 12, 0.66))
tp_data.new_estimate.sum()

86.98738404481048

In [152]:
tp_data.state_close_estimate.sum()

39.52315681719646

In [214]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')


In [212]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')

In [224]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [244]:
x = tp_data.state_chain.iloc[0][0]

tp_data.loc[(tp_data.state_chain[0] == 1)]

KeyError: 'False: boolean label can not be used without a boolean index'

In [247]:
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [250]:
tp_data.state_close_estimate.sum()

46.89175083298029

In [248]:
tp_data

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NFLX,2,2022-11-29 15:30:00,2022-11-30 11:15:00,281.168,283.248,-1.000000,2.354808,-1.000000,-1.000000,-1.000000,"[0.14326923076923273, 0.3596153846153828, 0.29...",24,5,0.01,1.139423,"[0.12573839662447417, 0.31561181434598967, 0.2..."
376,NFLX,1,2022-11-30 14:35:00,2022-11-30 14:40:00,283.605,281.780,-1.000000,-0.523288,-1.000000,-1.000000,-1.000000,"[-0.5232876712329024, -0.9178082191780942]",24,5,0.01,1.158904,"[-0.4515366430260266, -0.791962174940908]"
377,NFLX,1,2022-12-01 20:15:00,2022-12-01 20:20:00,318.462,317.632,-1.000000,-0.592771,-1.000000,-1.000000,-1.000000,"[-0.677108433734967, -0.5927710843373147]",24,5,0.01,1.349398,"[-0.501785714285732, -0.43928571428568647]"
378,NFLX,1,2022-12-02 20:25:00,2022-12-02 20:35:00,320.746,320.056,-1.000000,0.223188,-1.000000,-1.000000,-1.000000,"[0.22318840579701454, -0.09565217391308861, -0...",24,5,0.01,1.420290,"[0.1571428571427958, -0.06734693877554192, -0...."


In [157]:
path =  '/Users/tranthong/Desktop/stock-app/stock_data_M5/'
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = path + ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

start = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_start.iloc[0]
end = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_end.iloc[0]
ticker = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_ticker.iloc[0]
open = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_open.iloc[0]
stoploss = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_stop_loss.iloc[0]
r = abs(open -stoploss)
data =  get_mt5_raw_data_range(ticker, start, end)

In [158]:
(data.Close - open) / r

0       0.292308
1       0.523077
2      -0.015385
3       0.446154
4       0.984615
         ...    
297    18.215385
298    18.215385
299    18.215385
300    18.061538
301    17.138462
Name: Close, Length: 302, dtype: float64

# schedule
